In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

In [2]:
df = pd.read_json('/home/ashish/VScode files/Python files/projects/ML project/Loan-Approval-ML-project/notebook/data/loan_approval_dataset.json')

In [3]:
df.head()

,Id,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag
0,1,1303834,23,3,single,rented,no,Mechanical_engineer,Rewa,Madhya_Pradesh,3,13,0
1,2,7574516,40,10,single,rented,no,Software_Developer,Parbhani,Maharashtra,9,13,0
2,3,3991815,66,4,married,rented,no,Technical_writer,Alappuzha,Kerala,4,10,0
3,4,6256451,41,2,single,rented,yes,Software_Developer,Bhubaneswar,Odisha,2,12,1
4,5,5768871,47,11,single,rented,no,Civil_servant,Tiruchirappalli[10],Tamil_Nadu,3,14,1


In [4]:
df.drop(['Id'], inplace = True, axis = 1)

In [5]:
df

,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag
0,1303834,23,3,single,rented,no,Mechanical_engineer,Rewa,Madhya_Pradesh,3,13,0
1,7574516,40,10,single,rented,no,Software_Developer,Parbhani,Maharashtra,9,13,0
2,3991815,66,4,married,rented,no,Technical_writer,Alappuzha,Kerala,4,10,0
3,6256451,41,2,single,rented,yes,Software_Developer,Bhubaneswar,Odisha,2,12,1
4,5768871,47,11,single,rented,no,Civil_servant,Tiruchirappalli[10],Tamil_Nadu,3,14,1
...,...,...,...,...,...,...,...,...,...,...,...,...
251995,8154883,43,13,single,rented,no,Surgeon,Kolkata,West_Bengal,6,11,0
251996,2843572,26,10,single,rented,no,Army_officer,Rewa,Madhya_Pradesh,6,11,0
251997,4522448,46,7,single,rented,no,Design_Engineer,Kalyan-Dombivli,Maharashtra,7,12,0
251998,6507128,45,0,single,rented,no,Graphic_Designer,Pondicherry,Puducherry,0,10,0


In [6]:
X = df.drop(['Risk_Flag'], axis = 1)

In [7]:
X.head()

,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS
0,1303834,23,3,single,rented,no,Mechanical_engineer,Rewa,Madhya_Pradesh,3,13
1,7574516,40,10,single,rented,no,Software_Developer,Parbhani,Maharashtra,9,13
2,3991815,66,4,married,rented,no,Technical_writer,Alappuzha,Kerala,4,10
3,6256451,41,2,single,rented,yes,Software_Developer,Bhubaneswar,Odisha,2,12
4,5768871,47,11,single,rented,no,Civil_servant,Tiruchirappalli[10],Tamil_Nadu,3,14


In [8]:
y = df['Risk_Flag']

In [9]:
y.head()

0    0
1    0
2    0
3    1
4    1
Name: Risk_Flag, dtype: int64

In [10]:
le = LabelEncoder()
sc = StandardScaler()

In [11]:
for i in X:
  if X[i].dtype == 'object':
      X[i] = le.fit_transform(X[i])
  else:
      X[i] = sc.fit_transform(X[[i]])

In [12]:
X.head()

,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS
0,-1.283145,-1.579604,-1.180232,1,2,0,33,251,13,-0.914131,0.716356
1,0.895457,-0.583344,-0.014067,1,2,0,43,227,14,0.731036,0.716356
2,-0.349269,0.940348,-1.013637,0,2,0,47,8,12,-0.639936,-1.427981
3,0.437526,-0.524740,-1.346827,1,2,1,43,54,17,-1.188325,0.001577
4,0.268128,-0.173119,0.152528,1,2,0,11,296,22,-0.914131,1.431135


In [13]:
X.shape

(252000, 11)

In [14]:
y.shape

(252000,)

In [15]:
# seperate dataset into train and test

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 21)

In [16]:
X_train.shape, X_test.shape

((201600, 11), (50400, 11))

In [17]:
y_train.shape,y_test.shape

((201600,), (50400,))

In [18]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)

    return mae, rmse, r2_square

In [19]:
models = {
    "K-Neighbors Classifier": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest Classifier": RandomForestClassifier(),
    "XGBClassifier": XGBClassifier(),
    "CatBoosting Classifier": CatBoostClassifier(verbose=False),
    "AdaBoost Classifier": AdaBoostClassifier()
}

model_list = []
r2_list = []

for model_name, model in models.items():
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Evaluate Train and Test dataset
    model_train_score = model.score(X_train, y_train)
    model_test_score = model.score(X_test, y_test)

    print(model_name)
    model_list.append(model_name)

    print('Model performance for Training set')
    print("- Score: {:.4f}".format(model_train_score))

    print('----------------------------------')

    print('Model performance for Test set')
    print("- Score: {:.4f}".format(model_test_score))
    r2_list.append(model_test_score)

    print('='*35)
    print('\n')

K-Neighbors Classifier
Model performance for Training set
- Score: 0.9008
----------------------------------
Model performance for Test set
- Score: 0.8908


Decision Tree
Model performance for Training set
- Score: 0.9363
----------------------------------
Model performance for Test set
- Score: 0.8842


Random Forest Classifier
Model performance for Training set
- Score: 0.9363
----------------------------------
Model performance for Test set
- Score: 0.9005


XGBClassifier
Model performance for Training set
- Score: 0.8958
----------------------------------
Model performance for Test set
- Score: 0.8926


CatBoosting Classifier
Model performance for Training set
- Score: 0.8948
----------------------------------
Model performance for Test set
- Score: 0.8916




/home/ashish/VScode files/Python files/projects/ML project/Loan-Approval-ML-project/.venv/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoost Classifier
Model performance for Training set
- Score: 0.8764
----------------------------------
Model performance for Test set
- Score: 0.8800




In [20]:
df_results = pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)
df_results

,Model Name,R2_Score
2,Random Forest Classifier,0.900536
3,XGBClassifier,0.892579
4,CatBoosting Classifier,0.891587
0,K-Neighbors Classifier,0.890754
1,Decision Tree,0.884187
5,AdaBoost Classifier,0.880040
